# Hands-on Tutorial on XAI (Introduction)
### Machine Learning and Dynamical Systems Seminar hosted by the Alan Turing Institute

[Link](https://github.com/steveazzolin/gdl_tutorial_turinginst) to online material

![](./SCM_XAITutorial.png)

Given the Structural Causal Model defined above, we will sample some data from the underlying distribution and train two ML models with the goal of inspecting their behaviour under environment shifts. Then, by inspecting the learned weights, we try to judge whether the model learned the expected decision patterns

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

In [ ]:
def create_data_env(n_samples, mu_1, dev_2, mu_3):
  x1 = np.random.randn(n_samples, 1) + mu_1
  y = x1 >= mu_1
  x2 = y + np.random.randn(n_samples, 1) * dev_2
  x3 = np.random.randn(n_samples, 1) + mu_3

  X = np.concatenate([x1,x2,x3], axis=-1)
  y = np.squeeze(y)
  return X, y

Create data from 3 different environments

In [ ]:
n_samples = 1000
mu_1 = 5
mu_3 = 2.5
dev_2_env_1 = 0.1
dev_2_env_2 = 1
dev_2_env_3 = 10

X, y = {}, {}

X_env, y_env = create_data_env(n_samples, mu_1, dev_2_env_1, mu_3)
X["env_1"] = X_env
y["env_1"] = y_env

X_env, y_env = create_data_env(n_samples, mu_1, dev_2_env_2, mu_3)
X["env_2"] = X_env
y["env_2"] = y_env

X_env, y_env = create_data_env(n_samples, mu_1, dev_2_env_3, mu_3)
X["env_3"] = X_env
y["env_3"] = y_env


print(X["env_1"].shape, y["env_1"].shape)

(1000, 3) (1000,)


Fit a Simple **LogisticRegression** on the first environment

In [ ]:
clf = LogisticRegression(random_state=42).fit(X["env_1"], y["env_1"])
preds = clf.predict(X["env_1"])
acc = clf.score(X["env_1"], y["env_1"])
print("Accuracy of the model on env 1 = ", acc)

Accuracy of the model on env 1 =  1.0


Does it generalize to the other environments?

In [ ]:
acc = clf.score(X["env_2"], y["env_2"])
print("Accuracy of the model on env 2 = ", acc)

acc = clf.score(X["env_3"], y["env_3"])
print("Accuracy of the model on env 3 = ", acc)

Accuracy of the model =  0.839
Accuracy of the model =  0.535


Why? Can we inspect what it has learned?
The **LogisticRegression** is defined as:

$\hat{y} = \sigma(w_1x_1 + w_2x_2 + w_3x_3$)

So we can easily inspect the individual weights

In [ ]:
clf.coef_

array([[ 3.52944624,  5.63905447, -0.01044425]])

So, the model gave a large importance to $x_2$ even tough is not causally asssociated to $y$. This opens the model to several vulnerabilities associated to distribution/environment shifts.

Let's try with a Deep Learning model, specifically a **MLP** with 2 layers and 100 hidden units

In [ ]:
clf = MLPClassifier(random_state=42, max_iter=700).fit(X["env_1"], y["env_1"])
for e in ["env_1", "env_2", "env_3"]:
  acc = clf.score(X[e], y[e])
  print(f"Accuracy of the model on {e}= ", acc)

Accuracy of the model on env_1=  1.0
Accuracy of the model on env_2=  0.715
Accuracy of the model on env_3=  0.518


Let's inspect again the weights

In [ ]:
Ws = clf.coefs_
print("Layers of the network: ", len(Ws))
print("Num of params of first layer: ", Ws[0].shape)
print(Ws[0])

Layers of the network:  2
Num of params of first layer:  (3, 100)
[[-2.65515764e-09  2.57549541e-01  1.37231616e-01  6.30027767e-02
  -1.31288889e-02 -1.08151796e-01  1.75626295e-02  1.64818077e-01
  -1.94127872e-02  1.52594666e-01 -1.10490703e-03  2.39885965e-01
   1.80653317e-01  7.68429728e-02 -1.78669153e-01  1.34819530e-02
  -4.14915116e-07  4.66655897e-02  8.76425924e-02  4.70882182e-02
   7.82715752e-02 -4.27435978e-02  7.50783758e-02 -5.66060247e-09
   4.10681634e-02  1.96456209e-01 -1.15670590e-01  8.78970932e-02
   3.50042473e-02 -7.52995646e-04  3.44799913e-02 -5.81057339e-02
  -5.58299294e-04  2.00182986e-01  2.16427816e-01  1.68570033e-01
   3.28088392e-02 -4.89513496e-02  1.05677975e-01 -4.70809681e-02
  -1.87730461e-01  2.02433909e-02 -3.07742131e-02  1.68925265e-01
  -3.52543746e-06  8.70598701e-02 -3.66671499e-02  8.53423391e-02
   2.55239516e-02 -2.15168526e-02  2.49983136e-01  1.08575903e-01
   1.79477713e-01  2.18559381e-01  2.39604066e-03  1.42257260e-01
  -6.43890

With this deep learning model, even if failry shallow, we can't assess the goodness of the model by inspewcting the parameters, since theiy are simply too many.